# In Class Lab 1

### Due by midnight, thursday in your github repository 'Labs/Lab1' folder


In [6]:
# Import Modules 
import numpy as np # import numpy
import astropy.units as u # import astropy units
from astropy import constants as const # import astropy constants

### Astropy Units:  https://docs.astropy.org/en/stable/units/index.html


## Part A:  The Local Standard of Rest
Proper motion of Sgr A* from Reid & Brunthaler 2004
$\mu = 6.379$ mas/yr 

Peculiar motion of the sun, $v_\odot$ = 12.24 km/s  (Schonrich 2010)


$v_{tan} = 4.74 \frac{\mu}{\rm mas/yr} \frac{R_o}{\rm kpc} = V_{LSR} + v_\odot$


### a)

Create a function called VLSR to compute the local standard of res (V$_{LSR}$).

The function should take as input: the solar radius (R$_o$), the proper motion (mu)
and the peculiar motion of the sun in the $v_\odot$ direction.

Compute V$_{LSR}$ using three different values R$_o$: 
1. Water Maser Distance for the Sun :  R$_o$ = 8.34 kpc   (Reid 2014 ApJ 783) 
2. GRAVITY Collaboration Distance for the Sun:  R$_o$ = 8.178 kpc   (Abuter+2019 A&A 625)
3. Value for Distance to Sun listed in Sparke & Gallagher : R$_o$ = 7.9 kpc 


In [7]:
def VLSR(R0, mu=6.379*u.mas/u.yr, v_sun=12.24*u.km/u.s):
    '''
    Calculates the local standard of rest velocity of the sun.
    
    Arguments:
        R0: astropy.Quantity, solar radius, any unit of length
        mu: astropy.Quantity, proper motion, any unit of velocity
        v_sun: astropy.Quantity, peculiar motion of the sun, any unit of velocity
    
    Returns:
        VLSR: astropy.Quantity, VLSR in km/s
    '''
    R0 = R0.to(u.kpc) / u.kpc
    mu = mu.to(u.mas/u.yr) * u.yr/u.mas
    v_tan = 4.74*R0*mu*u.km/u.s
    VLSR = v_tan - v_sun
    
    return VLSR

In [8]:
# Calcuclate VLSR for each R0
VLSR_Reid = VLSR(8.34*u.kpc)
VLSR_Abuter = VLSR(8.178*u.kpc)
VLSR_SnG = VLSR(7.9*u.kpc)

In [9]:
print(f'Answer for 1: {VLSR_Reid}')
print(f'Answer for 2: {VLSR_Abuter}')
print(f'Answer for 3: {VLSR_SnG}')

Answer for 1: 239.9320764 km / s
Answer for 2: 235.03376988000002 km / s
Answer for 3: 226.628034 km / s


### b)

compute the orbital period of the sun in Gyr using R$_o$ from the GRAVITY Collaboration (assume circular orbit)

Note that 1 km/s $\sim$ 1kpc/Gyr

In [21]:
def calc_orbital_period(R0, VLSR):
    '''
    Calculates orbital period of the sun around the galaxy.
    
    Arguments:
        R0: astropy.Quantity, solar radius, any unit of length
        VLSR: astropy.Quantity, local standard of rest velocity, any unit of velocity
    
    Returns:
        T_sun: astropy.Quantity, orbital period in Gyrs
    '''
    R0 = R0.to(u.kpc)
    VLSR = VLSR.to(u.kpc/u.Gyr)
    T_sun = 2*np.pi*R0/VLSR
    
    return T_sun

In [26]:
# Using R0 from GRAVITY
R0_Abuter = 8.178*u.kpc
T_sun = calc_orbital_period(R0_Abuter, VLSR_Abuter)

In [23]:
print(f'orbital period of the sun: {T_sun}')

orbital period of the sun: 0.2137682914325781 Gyr


### c)

Compute the number of rotations about the GC over the age of the universe (13.8 Gyr)

In [24]:
AgeUniverse = 13.8*u.Gyr
N_rotations = AgeUniverse/T_sun
print(f'Sun rotates {N_rotations} times over the age of the universe')

Sun rotates 64.55587920696125 times over the age of the universe


## Part B  Dark Matter Density Profiles

### a)
Try out Fitting Rotation Curves 
[here](http://wittman.physics.ucdavis.edu/Animations/RotationCurve/GalacticRotation.html)


### b)


In the Isothermal Sphere model, what is the mass enclosed within the solar radius (R$_o$) in units of M$_\odot$? 

Recall that for the Isothermal sphere :
$\rho(r) = \frac{V_{LSR}^2}{4\pi G r^2}$

Where $G$ = 4.4985e-6 kpc$^3$/Gyr$^2$/M$_\odot$, r is in kpc and $V_{LSR}$ is in km/s

What about at 260 kpc (in units of  M$_\odot$) ? 

In [25]:
def massIso(r, VLSR):
    '''
    Computes the dark matter mass enclosed within a given radius,
    assuming an Isothermal Sphere Model M(r) = VLSR^2/G*r
    
    Arguments:
        r: astropy.Quantity, radius of the sphere enclosing the mass to be computed
        VLSR: astropy.Quantity, local standard of rest velocity
    
    Returns:
        M: astropy.Quantity, mass enclosed within r, in Msun
    '''
    G = 4.4985e-6 * (u.kpc**3/u.Gyr**2/u.Msun)
    r = r.to(u.kpc)
    VLSR = VLSR.to(u.kpc/u.Gyr)
    M = VLSR**2/G*r
    
    return M

In [31]:
# Calculate enclosed mass at solar radius
mIsoSolar = massIso(R0_Abuter, VLSR_Abuter)
print(f'{mIsoSolar:.2e}')

1.05e+11 solMass


In [32]:
# Calculate enclosed mass at 260 kpc
mIso260 = massIso(260*u.kpc, VLSR_Abuter)
print(f'{mIso260:.2e}')

3.34e+12 solMass


## c) 

The Leo I satellite is one of the fastest moving satellite galaxies we know. 


It is moving with 3D velocity of magnitude: Vtot = 196 km/s at a distance of 260 kpc (Sohn 2013 ApJ 768)

If we assume that Leo I is moving at the escape speed:

$v_{esc}^2 = 2|\Phi| = 2 \int G \frac{\rho(r)}{r}dV $ 

and assuming the Milky Way is well modeled by a Hernquist Sphere with a scale radius of $a$= 30 kpc, what is the minimum mass of the Milky Way (in units of M$_\odot$) ?  

How does this compare to estimates of the mass assuming the Isothermal Sphere model at 260 kpc (from your answer above)

In [34]:
def massHernVesc(vesc, r, a=30*u.kpc):
    '''
    Determines the total dark matter mass needed given an escape speed,
    assuming a Hernquist profile.
    
    Arguments:
        vesc: astropy.Quantity, escape speed, any unit of velocity
        r: astropy.Quantity, distance from galactic center, any unit of length
        a: astropy.Quantity, the Hernquist scale length, in kpc, default 30 kpc.
        
    Returns:
        M: astropy.Quantity, minimum mass of Milky Way, in Msun
    '''
    G = const.G.to(u.kpc**3/u.Gyr**2/u.Msun)
    vesc = vesc.to(u.kpc/u.Gyr)
    r = r.to(u.kpc)
    a = a.to(u.kpc)
    M = vesc**2/2/G*(r+a)
    
    return M

In [37]:
Vleo = 196*u.km/u.s
r = 260*u.kpc
M = massHernVesc(Vleo, r)
print(f'Mass of Dark matter to keep LeoI bound: {M:.2e}')

Mass of Dark matter to keep LeoI bound: 1.30e+12 solMass
